# Import library

In [2]:
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic') 

In [3]:
import os 
import sys
import pandas as pd
import numpy as np
import pickle

from glob import glob
from PIL import Image

import torch 
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import torch.nn as nn
from torch.optim import lr_scheduler
import torch.optim as optim
from torchvision.io import read_image

import matplotlib.pyplot as plt
import time
import copy
import random
from tqdm import tqdm

from joblib import dump, load

from sklearn import preprocessing 

In [4]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b0')

Loaded pretrained weights for efficientnet-b0


In [5]:
model_name = 'efficientnet-b0'
image_size = EfficientNet.get_image_size(model_name)
print(image_size)

224


In [6]:
model = EfficientNet.from_pretrained(model_name, num_classes=10)

Loaded pretrained weights for efficientnet-b0


# Load tofu datasets

In [114]:
class TofuDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file, encoding='cp949')
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform


    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)        
        if self.target_transform:
            label = self.target_transform.transform([label])[0]


        return image, label

In [115]:
def global_contrast_normalization(x: torch.tensor, scale='l2'):
    """
    Apply global contrast normalization to tensor, i.e. subtract mean across features (pixels) and normalize by scale,
    which is either the standard deviation, L1- or L2-norm across features (pixels).
    Note this is a *per sample* normalization globally across features (and not across the dataset).
    """

    assert scale in ('l1', 'l2')

    n_features = int(np.prod(x.shape))
    
    x = x.float()

    mean = torch.mean(x)  # mean over all features (pixels) per sample
    x -= mean

    if scale == 'l1':
        x_scale = torch.mean(torch.abs(x))

    if scale == 'l2':
        x_scale = torch.sqrt(torch.sum(x ** 2)) / n_features

    x /= x_scale

    return x

In [116]:
# Pre-computed min and max values (after applying GCN) from train data per class
min_ = - 4.4943
max_ = 2.3380

In [117]:
transform = transforms.Compose([
                                transforms.Lambda(lambda x: global_contrast_normalization(x, scale='l1')),
                                transforms.Normalize([min_],
                                                     [max_ - min_])
                                ])

In [118]:
folders = ['정상A', '정상B', '금속', '머리카락', '벌레', '상단불량D', '상단불량E', '유바', '탄화물', '플라스틱']
le = preprocessing.LabelEncoder()
le.fit(folders)

LabelEncoder()

In [119]:
le_name_mapping = dict(zip(le.transform(le.classes_),le.classes_))
le_name_mapping

{0: '금속',
 1: '머리카락',
 2: '벌레',
 3: '상단불량D',
 4: '상단불량E',
 5: '유바',
 6: '정상A',
 7: '정상B',
 8: '탄화물',
 9: '플라스틱'}

In [120]:
target_transform = le

In [121]:
le.transform(['정상A'])[0]

6

In [122]:
train_dataset = TofuDataset('../train_annotations.csv', '/workspace/TOFU_Kaier/DB/TOFU_BOX_margin/train', transform, target_transform )
valid_dataset = TofuDataset('../valid_annotations.csv', '/workspace/TOFU_Kaier/DB/TOFU_BOX_margin/valid', transform, target_transform)
test_dataset = TofuDataset('../test_annotations.csv', '/workspace/TOFU_Kaier/DB/TOFU_BOX_margin/test', transform, target_transform)

In [123]:
batch_size=64
dataloaders = {}
dataloaders['train'] = DataLoader(train_dataset, batch_size=batch_size,
                        shuffle=True, num_workers=0)
dataloaders['valid'] = DataLoader(valid_dataset, batch_size=batch_size,
                        shuffle=True, num_workers=0)
dataloaders['test'] = DataLoader(test_dataset, batch_size=batch_size,
                        shuffle=False, num_workers=0)

In [124]:
#check dataloader shape
for img, label in dataloaders['train']:
    print(img.shape)
    print(label)
    break

torch.Size([64, 3, 640, 640])
tensor([6, 6, 2, 0, 2, 7, 6, 6, 3, 4, 0, 8, 6, 7, 9, 1, 2, 6, 6, 0, 6, 0, 7, 3,
        6, 6, 7, 7, 3, 6, 4, 6, 6, 6, 7, 6, 6, 4, 7, 7, 0, 1, 6, 8, 7, 7, 4, 7,
        3, 6, 6, 6, 7, 9, 7, 6, 7, 6, 3, 6, 6, 8, 6, 3])


# Efficientnet transfer learning

In [125]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    train_loss, train_acc, valid_loss, valid_acc = [], [], [], []
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss, running_corrects, num_cnt = 0.0, 0, 0
            
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                num_cnt += len(labels)
            if phase == 'train':
                scheduler.step()
            
            epoch_loss = float(running_loss / num_cnt)
            epoch_acc  = float((running_corrects.double() / num_cnt).cpu()*100)
            
            if phase == 'train':
                train_loss.append(epoch_loss)
                train_acc.append(epoch_acc)
            else:
                valid_loss.append(epoch_loss)
                valid_acc.append(epoch_acc)
            print('{} Loss: {:.2f} Acc: {:.1f}'.format(phase, epoch_loss, epoch_acc))
           
            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_idx = epoch
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
#                 best_model_wts = copy.deepcopy(model.module.state_dict())
                print('==> best model saved - %d / %.1f'%(best_idx, best_acc))

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best valid Acc: %d - %.1f' %(best_idx, best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), 'GCN_efficientnet_b0_president_model.pt')
    print('model saved')
    return model, best_idx, best_acc, train_loss, train_acc, valid_loss, valid_acc

In [126]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # set gpu

model = model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.Adam(model.parameters(), 
                         lr = 1e-4)

lmbda = lambda epoch: 0.98739
exp_lr_scheduler = optim.lr_scheduler.MultiplicativeLR(optimizer_ft, lr_lambda=lmbda)


In [127]:
model, best_idx, best_acc, train_loss, train_acc, valid_loss, valid_acc = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=100)

Epoch 0/99
----------


RuntimeError: CUDA out of memory. Tried to allocate 2.34 GiB (GPU 0; 15.75 GiB total capacity; 12.75 GiB already allocated; 1.91 GiB free; 12.77 GiB reserved in total by PyTorch)

In [ ]:
## 결과 그래프 그리기
print('best model : %d - %1.f / %.1f'%(best_idx, valid_acc[best_idx], valid_loss[best_idx]))
fig, ax1 = plt.subplots()

ax1.plot(train_acc, 'b-')
ax1.plot(valid_acc, 'r-')
plt.plot(best_idx, valid_acc[best_idx], 'ro')
ax1.set_xlabel('epoch')
# Make the y-axis label, ticks and tick labels match the line color.
ax1.set_ylabel('acc', color='k')
ax1.tick_params('y', colors='k')

ax2 = ax1.twinx()
ax2.plot(train_loss, 'g-')
ax2.plot(valid_loss, 'k-')
plt.plot(best_idx, valid_loss[best_idx], 'ro')
ax2.set_ylabel('loss', color='k')
ax2.tick_params('y', colors='k')

fig.tight_layout()
plt.show()

In [ ]:
# load saved model weights
model_path = '/content/drive/MyDrive/Kaier_Tofu_Data/Regional attention/raw_data_president_model.pt'
model_dict = torch.load(model_path)
model.load_state_dict(model_dict)

<All keys matched successfully>

In [ ]:
def test(model, phase = 'test'):
    # phase = 'train', 'valid', 'test'
    
    model.eval()
    
    running_loss, running_corrects, num_cnt = 0.0, 0, 0

    with torch.no_grad():
        predict_label_list = []
        for i, (inputs, labels) in tqdm(enumerate(dataloaders[phase])):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)  # batch의 평균 loss 출력

            running_loss    += loss.item() * inputs.size(0)
            running_corrects+= torch.sum(preds == labels.data)
            num_cnt += inputs.size(0)  # batch size

            predict_label_list += (preds.cpu().tolist())
  
          

            # if i == 2: break

        test_loss = running_loss / num_cnt
        test_acc  = running_corrects.double() / num_cnt       
        print('test done : loss/acc : %.2f / %.1f' % (test_loss, test_acc*100))

    return predict_label_list

In [ ]:
predict_label_list = test(model, phase = 'test')

131it [00:15,  8.28it/s]

test done : loss/acc : 1.05 / 87.2


In [ ]:
# class별 accuracy 
class_names = ['정상A', '정상B', '금속', '머리카락', '벌레', '상단불량D', '상단불량E', '유바', '탄화물', '플라스틱']
class_accuracy = dict()
for class_ in class_names:
  idx = np.in1d(test_class, class_)
  class_tmp = (test_label==predict_label_list)[idx]
  accruacy_ = np.sum(class_tmp) / len(class_tmp)
  class_accuracy[class_] = accruacy_
class_accuracy
  

{'금속': 0.77,
 '머리카락': 0.9923076923076923,
 '벌레': 0.88,
 '상단불량D': 0.6784660766961652,
 '상단불량E': 0.9344262295081968,
 '유바': 0.9166666666666666,
 '정상A': 0.9237113402061856,
 '정상B': 0.8643649815043156,
 '탄화물': 0.7295081967213115,
 '플라스틱': 0.89}

In [ ]:
le_name_mapping = dict(zip(le.transform(le.classes_),le.classes_))
le_name_mapping

{0: '금속',
 1: '머리카락',
 2: '벌레',
 3: '상단불량D',
 4: '상단불량E',
 5: '유바',
 6: '정상A',
 7: '정상B',
 8: '탄화물',
 9: '플라스틱'}

In [ ]:
predict_class_list = [le_name_mapping[x] for x in predict_label_list]
test_class_list = [le_name_mapping[x] for x in test_label]


In [ ]:
!pip install scikit-plot

In [ ]:
img.shape

torch.Size([64, 3, 224, 224])

In [ ]:
print(img.shape)
features = model.extract_features(img.cuda())
print(features.shape)  

torch.Size([64, 3, 224, 224])
torch.Size([64, 1280, 7, 7])
